In [2]:

import json
from pandas import json_normalize

data_infile = 'open-data/data/events/'
match_id = 7298
data_path = data_infile+ str(match_id) + '.json'


with open(data_path) as f:
    data = json.load(f)

In [3]:



players = {}

for event in data: 
    if event.get('type', {}).get('name') == 'Starting XI': 
        lineup = event.get('tactics', {}).get('lineup', [])
        for entry in lineup: 
            player = entry['player']
            players[player['name']] = player['id']


In [4]:
def shot_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    outcome = event.get('interception', {}).get('outcome', '')
    return f"Shot py {player} of team{team} from {start_location}, {outcome}"
    
def offside_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    return f"{player} of {team} caught offsides"

def interception_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    outcome = event.get('interception', {}).get('outcome', '')
    return f"Interception {outcome} by {player} of {team} at {start_location}" 
    
def foul_won_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f'Foul won by {player} of team {team} at {start_location}'

def foul_committed_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f'{player} of team {team} committed foul at {start_location}'
def dribbled_past(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f'{player} of team {team} dribbled past at {start_location}'

def dribble_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    outcome = event.get('dribble', '').get('outcome', '')
    return f'{outcome} dribble by {player} of team {team} at {start_location}'

def dispossed_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f"{player} of team {team} dispossed at {start_location}"

def clearance_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    play_pattern = event.get('location', [])
    return f'Clearance by {player} of team {team} from {start_location} in {play_pattern}'

def block_event(event):
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    play_pattern = event.get('play_pattern').get('name', '')
    return f"Block by {player} of team {team} at {start_location} from a {play_pattern}"

def carry_event(event):
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    play_pattern = event.get('play_pattern').get('name', '')
    duration = event.get('duration', 0)
    end_location = event.get('carry', {}).get('end_location', [])
    return f"Carry by {player} of team {team} from {start_location} to {end_location} lasting {duration}"

def pass_event(event):
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    pass_details = event.get('pass', {})
    pass_type = pass_details.get('height', {}).get('name', 'Ground Pass')
    length = round(pass_details.get('length', 0), 1)
    start_location = event.get('location', [])
    end_location = pass_details.get('end_location', [])
    under_pressure = event.get('under_pressure', False)
    pressure_note = " under pressure" if under_pressure else ""

    recipient = pass_details.get('recipient', {}).get('name')
    outcome = pass_details.get('outcome', {}).get('name')  # e.g., 'Incomplete'

    if recipient and not outcome:
        return f"{player} of {team} makes a {length}m {pass_type.lower()} to {recipient} from {start_location} to {end_location}{pressure_note}."
    elif outcome:
        return f"{player} of {team} attempts a {length}m {pass_type.lower()} from {start_location}{pressure_note}, but it is {outcome.lower()}."
    else:
        return f"{player} of {team} plays a {length}m {pass_type.lower()} from {start_location} to {end_location}{pressure_note}."
    
def ball_recovery(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])

    return f"Ball Recovery by {player} of team {team} at {start_location}"


In [5]:
EVENT_DISPATCH = {
    'Shot': shot_event,
    'Offside': offside_event,
    'Interception': interception_event,
    'Foul Won': foul_won_event,
    'Foul Committed': foul_committed_event,
    'Dribbled Past': dribbled_past,
    'Dribble': dribble_event,
    'Dispossessed': dispossed_event,
    'Clearance': clearance_event,
    'Block': block_event,
    'Carry': carry_event, 
    'Pass': pass_event
}


In [6]:
def process_event(event):
    event_type = event.get('type', {}).get('name')
    formatter = EVENT_DISPATCH.get(event_type)
    if formatter:
        return formatter(event)
    return None  


In [7]:
prompts = []
for event in data:
    prompt = process_event(event)
    if prompt: 
        prompt = "I am going to describe a football event and I want you to generate a piece of commentary for that event: " + prompt
        prompts.append(prompt)
    
print(prompts[0])


I am going to describe a football event and I want you to generate a piece of commentary for that event: So-Yun Ji of Chelsea FCW makes a 3.6m ground pass to Ramona Bachmann from [61.0, 40.0] to [63.0, 37.0].


In [8]:
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct")




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use mps:0


In [ ]:
from transformers import pipeline

import torch

pipe = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float16,                    
    device_map="auto"                             
)

messages = [{"role": "user", "content": "Generate a short piece of football commentary for: So-Yun Ji of Chelsea FCW makes a 3.6m pass to Ramona Bachmann."}]
print(pipe(messages)[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use mps
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/Users/mattschaefer/opt/anaconda3/envs/bayes/lib/python3.10/site-packages/transformers/pytorch_utils.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


[{'role': 'user', 'content': 'Generate a short piece of football commentary for: So-Yun Ji of Chelsea FCW makes a 3.6m pass to Ramona Bachmann.'}, {'role': 'assistant', 'content': ' "Wow, what a fantastic pass from So-Yun Ji of Chelsea FCW! The '}]
